In [25]:
!pip install -q torch
#!pip install -q openai moviepy
#!pip install SpeechRecognition
!pip install -q transformers
!pip install -q datasets
!pip install -q qdrant_client
#!pip uninstall nltk
!pip install -U nltk

In [26]:
import nltk
#nltk.download('punkt', force=True)
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('punkt')
#import moviepy.editor as mp
import os
import glob
#import speech_recognition as sr
import csv
import numpy as np
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.http import models
from transformers import AutoModel, AutoTokenizer
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
gsheetkey = "1Fp1T2Q1K-DVElVS-ta_QgqKj27y5wP88oC5p1O9gypA"

#sheet name
sheet_name = 'Data Baru'

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
df = pd.read_excel(url,sheet_name=sheet_name)
df.head()

,ID Anjing,ID Shelter,Name,Gender,ID Jenis,Jenis Anjing,Approximate Age,Sterilization Status,Rescue Story
0,1,1,Budi,Male,1,Indonesian Street Dog,2,Not Sterilized,Budi was found abandoned on the streets and wa...
1,2,2,Siti,Female,2,Indonesian Pomeranian,4,Spayed,Siti was rescued from a neglectful owner who c...
2,3,1,Raja,Male,3,Indonesian Mastiff,3,Neutered,Raja was saved from a dangerous situation and ...
3,4,2,Luna,Female,4,Indonesian Bali Dog,1,Not Sterilized,"Luna was found as a tiny puppy,abandoned in a ..."
4,5,2,Ciko,Male,5,Indonesian Dachshund,5,Not Sterilized,"Ciko was rescued from a puppy mill,and now he'..."


In [28]:
gsheetkey = "1cRSp9v-K2pyx_ArVDGM4yaZCgjfofXkIhnA4aDjzYr4"

#sheet name
sheet_name = 'DATA ONBOARDING'

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
df2 = pd.read_excel(url,sheet_name=sheet_name)
df2.head()


,user_id,group,age,sterilisasi,rescue_story_1,rescue_story_2,rescue_story_3,request_story
0,1,Herding Group,2,Not Sterilized,Daisy was rescued from a life-threatening situ...,Nina was rescued from an overcrowded shelter a...,"Winston, plucked from the tempestuous clutches...",Looking for a lively companion who loves outdo...
1,2,Toy Group,3,Spayed,"Surviving abandonment, Mocha, the resilient Go...","Rescued from hardship, Venus, a gentle Mastiff...",Riley's story is one of resilience as he bounc...,"Sweet and playful, perfect for a family with y..."
2,3,Working Group,1,Not Sterilized,"Charlie,the Labrador Retriever,was rescued fro...",Lola was rescued from a life of hardship and i...,"Max,the Poodle,was rescued from a neglectful o...","Intelligent and obedient, ideal for someone se..."
3,4,Terrier Group,1,Not Sterilized,"Rescued from a forest fire, Kiwi, the adventur...","Overcoming abandonment, Bruce, the spirited Sh...","At the shelter, Bailey awaits, prepared to bec...","Energetic and feisty, a great fit for an activ..."
4,5,Hound Group,2,Spayed,"Sasha the Dalmatian, a bundle of joy with a co...","Surviving adversity, Yuki, a dainty Schnauzer ...","Before being rescued, Leo faced uncertainty, s...","Gentle and affectionate, would make a wonderfu..."


In [29]:
client = QdrantClient(":memory:")

In [30]:
my_collection = "text_collection"
client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)
)

True

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModel.from_pretrained('gpt2')#.to(device) # switch this for GPU

In [32]:
def get_embeddings(row):
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # Menggabungkan beberapa kolom teks menjadi satu teks
    combined_text = ' '.join(row[['rescue_story_1', 'rescue_story_2', 'rescue_story_3', 'request_story']].astype(str).values)

    # Tokenisasi teks gabungan
    inputs = tokenizer(combined_text, padding=True, truncation=True, max_length=128, return_tensors="pt")

    # Matikan perhitungan gradien untuk operasi selanjutnya.
    with torch.no_grad():
        outputs = model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy()

    # Mengembalikan hasil embeddings yang dihitung.
    return outputs


In [33]:
df2['embeddings'] = df2.apply(get_embeddings, axis=1)
np.save("vectors", np.array(df2['embeddings']))

In [34]:
df2.head()

,user_id,group,age,sterilisasi,rescue_story_1,rescue_story_2,rescue_story_3,request_story,embeddings
0,1,Herding Group,2,Not Sterilized,Daisy was rescued from a life-threatening situ...,Nina was rescued from an overcrowded shelter a...,"Winston, plucked from the tempestuous clutches...",Looking for a lively companion who loves outdo...,"[[0.05293537, 0.0020674763, -0.69383526, 0.085..."
1,2,Toy Group,3,Spayed,"Surviving abandonment, Mocha, the resilient Go...","Rescued from hardship, Venus, a gentle Mastiff...",Riley's story is one of resilience as he bounc...,"Sweet and playful, perfect for a family with y...","[[-0.055041593, -0.11580643, -0.46906334, 0.15..."
2,3,Working Group,1,Not Sterilized,"Charlie,the Labrador Retriever,was rescued fro...",Lola was rescued from a life of hardship and i...,"Max,the Poodle,was rescued from a neglectful o...","Intelligent and obedient, ideal for someone se...","[[-0.034485284, 0.027605927, -0.21117806, 0.20..."
3,4,Terrier Group,1,Not Sterilized,"Rescued from a forest fire, Kiwi, the adventur...","Overcoming abandonment, Bruce, the spirited Sh...","At the shelter, Bailey awaits, prepared to bec...","Energetic and feisty, a great fit for an activ...","[[-0.016845975, -0.11060758, -0.49695095, -0.0..."
4,5,Hound Group,2,Spayed,"Sasha the Dalmatian, a bundle of joy with a co...","Surviving adversity, Yuki, a dainty Schnauzer ...","Before being rescued, Leo faced uncertainty, s...","Gentle and affectionate, would make a wonderfu...","[[0.11730222, -0.04319733, -0.75813943, -0.018..."


In [35]:
payload = df2[['group', 'age', 'sterilisasi', 'embeddings']].to_dict(orient="records")

In [36]:
# Set the expected size for the vector embeddings
expected_vector_size = 768

# Check if the tokenizer has a padding token, and if not, set it to the eos_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Define a function for mean pooling of token embeddings
def mean_pooling(model_output, attention_mask):
    # Extract token embeddings from the model output
    token_embeddings = model_output[0]

    # Expand the attention mask to match the size of token embeddings
    input_mask_expanded = (attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float())

    # Calculate the sum of token embeddings, considering the attention mask
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

    # Calculate the sum of the attention mask (clamped to avoid division by zero)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Return the mean-pooled embeddings
    return sum_embeddings / sum_mask

# Initialize a list to store text embeddings
text_embeddings = []

# Loop through each transcript in the 'Rescue Story' column of the 'df' variable
for transcript in df2['request_story']:
    # Tokenize the transcript, ensuring padding and truncation, and return PyTorch tensors
    inputs = tokenizer(transcript, padding=True, truncation=True, max_length=128, return_tensors="pt")

    # Perform inference using the model with the tokenized inputs
    with torch.no_grad():
        embs = model(**inputs)

    # Calculate mean-pooled embeddings using the defined function
    embedding = mean_pooling(embs, inputs["attention_mask"])

    # Ensure the embeddings are of the correct size by trimming or padding
    embedding = embedding[:, :expected_vector_size]

    # Append the resulting embedding to the list
    text_embeddings.append(embedding)


In [37]:
ids = list(range(len(df2)))

# Convert PyTorch tensors to lists of floats
text_embeddings_list = [[float(num) for num in emb.numpy().flatten().tolist()[:expected_vector_size]] for emb in text_embeddings]

client.upsert(collection_name=my_collection,
              points=models.Batch(
                  ids=ids,
                  vectors=text_embeddings_list,
                  payloads=payload
                  )
              )

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [38]:
from textblob import TextBlob

def calculate_sentiment_score(text):
    # Create a TextBlob object
    blob = TextBlob(text)

    # Get the sentiment polarity (-1 to 1, where -1 is negative, 0 is neutral, and 1 is positive)
    sentiment_score = blob.sentiment.polarity

    return sentiment_score


# Example usage:
text_example = df2['request_story'].iloc[len(df2) - 1]
sentiment_score_example = calculate_sentiment_score(text_example)
print(f"Sentiment Score: {sentiment_score_example}")

Sentiment Score: 0.30833333333333335


In [39]:
df2['Sentiment Score'] = df2['request_story'].apply(calculate_sentiment_score)
df2.head()

,user_id,group,age,sterilisasi,rescue_story_1,rescue_story_2,rescue_story_3,request_story,embeddings,Sentiment Score
0,1,Herding Group,2,Not Sterilized,Daisy was rescued from a life-threatening situ...,Nina was rescued from an overcrowded shelter a...,"Winston, plucked from the tempestuous clutches...",Looking for a lively companion who loves outdo...,"[[0.05293537, 0.0020674763, -0.69383526, 0.085...",0.136364
1,2,Toy Group,3,Spayed,"Surviving abandonment, Mocha, the resilient Go...","Rescued from hardship, Venus, a gentle Mastiff...",Riley's story is one of resilience as he bounc...,"Sweet and playful, perfect for a family with y...","[[-0.055041593, -0.11580643, -0.46906334, 0.15...",0.483333
2,3,Working Group,1,Not Sterilized,"Charlie,the Labrador Retriever,was rescued fro...",Lola was rescued from a life of hardship and i...,"Max,the Poodle,was rescued from a neglectful o...","Intelligent and obedient, ideal for someone se...","[[-0.034485284, 0.027605927, -0.21117806, 0.20...",0.608333
3,4,Terrier Group,1,Not Sterilized,"Rescued from a forest fire, Kiwi, the adventur...","Overcoming abandonment, Bruce, the spirited Sh...","At the shelter, Bailey awaits, prepared to bec...","Energetic and feisty, a great fit for an activ...","[[-0.016845975, -0.11060758, -0.49695095, -0.0...",0.340606
4,5,Hound Group,2,Spayed,"Sasha the Dalmatian, a bundle of joy with a co...","Surviving adversity, Yuki, a dainty Schnauzer ...","Before being rescued, Leo faced uncertainty, s...","Gentle and affectionate, would make a wonderfu...","[[0.11730222, -0.04319733, -0.75813943, -0.018...",0.600000


In [40]:
df2['avg_embeddings'] = df2['embeddings'].apply(lambda x: np.mean(x, axis=0))
df2['Opinion_Score'] = 0.7 * df2['avg_embeddings'] + 0.3 * df2['Sentiment Score']

In [41]:
df2.head()

,user_id,group,age,sterilisasi,rescue_story_1,rescue_story_2,rescue_story_3,request_story,embeddings,Sentiment Score,avg_embeddings,Opinion_Score
0,1,Herding Group,2,Not Sterilized,Daisy was rescued from a life-threatening situ...,Nina was rescued from an overcrowded shelter a...,"Winston, plucked from the tempestuous clutches...",Looking for a lively companion who loves outdo...,"[[0.05293537, 0.0020674763, -0.69383526, 0.085...",0.136364,"[0.05293537, 0.0020674763, -0.69383526, 0.0851...","[0.077963844, 0.042356323, -0.44477558, 0.1005..."
1,2,Toy Group,3,Spayed,"Surviving abandonment, Mocha, the resilient Go...","Rescued from hardship, Venus, a gentle Mastiff...",Riley's story is one of resilience as he bounc...,"Sweet and playful, perfect for a family with y...","[[-0.055041593, -0.11580643, -0.46906334, 0.15...",0.483333,"[-0.055041593, -0.11580643, -0.46906334, 0.153...","[0.10647088, 0.063935496, -0.18334435, 0.25217..."
2,3,Working Group,1,Not Sterilized,"Charlie,the Labrador Retriever,was rescued fro...",Lola was rescued from a life of hardship and i...,"Max,the Poodle,was rescued from a neglectful o...","Intelligent and obedient, ideal for someone se...","[[-0.034485284, 0.027605927, -0.21117806, 0.20...",0.608333,"[-0.034485284, 0.027605927, -0.21117806, 0.206...","[0.1583603, 0.20182416, 0.03467536, 0.3270195,..."
3,4,Terrier Group,1,Not Sterilized,"Rescued from a forest fire, Kiwi, the adventur...","Overcoming abandonment, Bruce, the spirited Sh...","At the shelter, Bailey awaits, prepared to bec...","Energetic and feisty, a great fit for an activ...","[[-0.016845975, -0.11060758, -0.49695095, -0.0...",0.340606,"[-0.016845975, -0.11060758, -0.49695095, -0.02...","[0.09038963, 0.024756514, -0.24568385, 0.08529..."
4,5,Hound Group,2,Spayed,"Sasha the Dalmatian, a bundle of joy with a co...","Surviving adversity, Yuki, a dainty Schnauzer ...","Before being rescued, Leo faced uncertainty, s...","Gentle and affectionate, would make a wonderfu...","[[0.11730222, -0.04319733, -0.75813943, -0.018...",0.600000,"[0.11730222, -0.04319733, -0.75813943, -0.0182...","[0.26211154, 0.14976187, -0.35069758, 0.167202..."


In [48]:
def get_recommendations(anjing):
    # Find the row corresponding to the given movie name
    query_row = df2[df2['request_story'] == anjing]  # Fix: use 'movie_name' instead of 'Name'

    if not query_row.empty:
        # Convert the 'Opinion_Score' column to a NumPy array
        opinion_scores_array = np.array(df2['Opinion_Score'].tolist())
        # Upsert the 'Opinion_Score' vectors to the Qdrant collection
        opinion_scores_ids = list(range(len(df2)))
        # Convert the 'Opinion_Score' array to a list of lists
        opinion_scores_list = opinion_scores_array.reshape(-1, 1).tolist()

        client.upsert(
            collection_name=my_collection,
            points=models.Batch(
                ids=opinion_scores_ids,
                vectors=opinion_scores_list
            )
        )
        # Define a query vector based on the opinion score you want to find similar movies for
        query_opinion_score = np.array([0.8] * 768)  # Adjust as needed

        # Perform a similarity search
        search_results = client.search(
            collection_name=my_collection,
            query_vector=query_opinion_score.tolist(),
            limit=10
        )

        # Extract movie recommendations from search results
        recommended_anjing_ids = [result.id for result in search_results]
        recommended_anjing = df2.loc[df2.index.isin(recommended_anjing_ids)]

        # Display recommended movies
        print("Recommended Result:")
        print(recommended_anjing[['rescue_story_1', 'rescue_story_2', 'rescue_story_3']])  # Fix: use 'recommended_movies' instead of 'recommended_pet'
    else:
        print(f"Anjing '{anjing}' not found in the dataset.")




In [49]:
# Example usage:
get_recommendations("Energetic and feisty, a great fit for an active and lively household.")

Recommended Result:
                                       rescue_story_1  \
0   Daisy was rescued from a life-threatening situ...   
16  In his quest for a family, Reggie, the fluffy ...   
17  Surviving neglect, Dolly, the gentle Boxer, ex...   
18  Ginger, a resilient Shih Tzu, conquered a diff...   
21  Sophie was rescued from a life-threatening sit...   
22  Ruby's rescue story is one of determination as...   
23  From street life to loving home, Rocco, the cu...   
25  Meet Waffles, the regal Rhodesian Ridgeback, i...   
27  Lola was rescued from a life of hardship and i...   
48  Charlie,the Labrador Retriever,was rescued fro...   

                                       rescue_story_2  \
0   Nina was rescued from an overcrowded shelter a...   
16  Rescued from the neglect of an uncertain past,...   
17  Once facing adversity, Kona, a gentle Beagle w...   
18  Dixie overcame neglect and now thrives in a ho...   
21  Ruby was rescued from a harsh environment and ...   
22  Lucky,